In [23]:
# Importing necessary libraries
import pandas as pd
import json
import sqlite3
import os

# Ensure the output directory exists
output_dir = 'outputs/project'
os.makedirs(output_dir, exist_ok=True)

# Function to fetch data from local file (CSV or JSON)
def fetch_data(source, file_type='csv'):
    try:
        if file_type == 'csv':
            return pd.read_csv(source)
        elif file_type == 'json':
            with open(source, 'r') as file:
                return pd.DataFrame(json.load(file))
    except Exception as e:
        print(f"Error in fetching data: {e}")
        return None

# Function to convert data between CSV and JSON
def convert_data(df, output_format='csv'):
    try:
        if output_format == 'csv':
            return df.to_csv(index=False)
        elif output_format == 'json':
            return df.to_json(orient='records')
    except Exception as e:
        print(f"Error in converting data: {e}")
        return None

# Function to add/remove columns from the dataset
def modify_columns(df, columns_to_add=None, columns_to_remove=None):
    try:
        if columns_to_add:
            for col_name, col_data in columns_to_add.items():
                df[col_name] = col_data
        if columns_to_remove:
            df.drop(columns=columns_to_remove, inplace=True)
        return df
    except Exception as e:
        print(f"Error in modifying columns: {e}")
        return None

#4.  Function to store data in SQL database or locally as a file
def store_data(df, output_file=None, sql_db=None, table_name=None):
    try:
        if output_file:
            if output_file.endswith('.csv'):
                df.to_csv(output_file, index=False)
            elif output_file.endswith('.json'):
                with open(output_file, 'w') as file:
                    json.dump(json.loads(df.to_json(orient='records')), file)
        if sql_db:
            conn = sqlite3.connect(sql_db)
            df.to_sql(table_name, conn, if_exists='replace', index=False)
            conn.close()
    except Exception as e:
        print(f"Error in storing data: {e}")

# Function to generate summary of data
def generate_summary(df):
    try:
        summary = {
            'num_records': len(df),
            'num_columns': len(df.columns)
        }
        return summary
    except Exception as e:
        print(f"Error in generating summary: {e}")
        return None

# 1. Ask the user for the input file path and format
input_file_path = input("Enter the path of the input file (CSV/JSON): ")
input_file_type = input("Enter the input file format (csv/json): ").lower()

# Fetch the input data
df = fetch_data(input_file_path, file_type=input_file_type)

# Generating initial summary
initial_summary = generate_summary(df)

# 3. Modifying the dataset (example: adding a new column)
if df is not None:
    modified_df = modify_columns(df, columns_to_add={'New_Col': [1]*len(df)})

# Generating final summary after modifications
final_summary = generate_summary(modified_df)

# 2. Ask the user for desired output format (CSV or JSON)
output_format = input("Enter the desired output file format (csv/json): ").lower()

# 4. storeCheck if conversion is necessary and convert the data if required
if input_file_type != output_format:
    # Convert the data to the desired output format
    converted_data = convert_data(modified_df, output_format)
    
    # Saving the converted data
    output_file_path = os.path.join(output_dir, f'modified_output.{output_format}')
    
    with open(output_file_path, 'w') as file:
        file.write(converted_data)
else:
    # If no conversion is needed, just save it in the same format
    output_file_path = os.path.join(output_dir, f'modified_output.{output_format}')
    store_data(modified_df, output_file=output_file_path)

# Output the final summaries
print({
    'initial_summary': initial_summary,
    'final_summary': final_summary,
    'output_file_path': output_file_path
})


{'initial_summary': {'num_records': 583, 'num_columns': 25}, 'final_summary': {'num_records': 583, 'num_columns': 26}, 'output_file_path': 'outputs/project/modified_output.json'}


## Reflection

The goal of this project was to implement an ETL (Extract, Transform, Load) data processor capable of ingesting either CSV or JSON files, allowing users to modify the data, and converting the data between these formats as per the user’s requirements. The final data could be stored locally as a file in the desired format, as well as optionally saved to an SQLite database.

Dynamic Input and Output Handling: One of the biggest challenges was dynamically handling different file formats (CSV/JSON). This required creating flexible functions to fetch and store data, as well as ensuring smooth conversion between formats. Ensuring that both CSV and JSON input could be processed correctly, and then converting between them, introduced complexity in the data manipulation logic.

Handling Edge Cases: Another challenge involved dealing with edge cases, such as malformed JSON files, missing columns in CSVs, or large datasets. It was crucial to provide meaningful error messages and ensure that the code could handle such cases gracefully without crashing.

User-Friendly Design: Designing the code to be user-friendly required a balance between functionality and simplicity. The user needed to provide input and output formats and file paths without being overwhelmed by technical details. Ensuring that the user interactions were clear and concise took careful planning.

What Was Easier Than Expected:
Using Pandas for Data Manipulation: Pandas proved to be a powerful tool for reading, modifying, and writing data. Its ability to seamlessly handle both CSV and JSON data formats made the development process more straightforward. Once the data was loaded into a Pandas DataFrame, modifying and converting it was relatively easy.

Directory Management and File Handling: Managing file paths and directories for storing outputs was simpler than anticipated. Python’s os.makedirs function ensured that output directories were created automatically if they did not exist, reducing manual setup.

What Was Harder Than Expected:
Handling Different Data Types for SQLite: Storing JSON-like structures into an SQLite database presented challenges. SQLite doesn't support certain complex data types (like lists or dictionaries), so converting these structures into supported formats (e.g., strings) before storing them required additional logic.

Error Handling and Validation: Ensuring proper validation of user input (e.g., checking if the file path is valid or if the format is correct) and implementing robust error handling across various scenarios took more effort. It was important to anticipate possible failures and provide informative feedback to users.

Usefulness for Future Data Projects:
This ETL utility is a versatile tool that can be incredibly useful in future data projects for the following reasons:

Flexible Data Ingestion: The ability to ingest data from different formats (CSV or JSON) and convert between them makes it useful for projects requiring data integration from multiple sources.
Data Transformation: The utility allows easy manipulation of the dataset (e.g., adding/removing columns), which is a common requirement in data cleaning and transformation tasks.
Automation of Repetitive Tasks: This utility automates the process of loading, transforming, and storing data, reducing manual effort. It can be extended to work with different file formats, databases, or even APIs.
Streamlined Workflow: By enabling flexible output formats and allowing modifications, this tool streamlines the data preprocessing pipeline, making it easier to integrate into larger data workflows.

Overall, this project provided valuable insights into building a flexible, user-friendly ETL pipeline. The challenges faced reinforced the importance of robust error handling and flexibility when dealing with diverse data sources and formats. This utility has broad applications in many data-related tasks, including data cleaning, transformation, and integration in future projects.